In [1]:
def tabamortissement(C, n, r, type, periodicite, differe=1, q=1.0):
    
    import pandas as pd
    import numpy as np
    
    # ÉTAPE 1 : INITIALISATION ET CONVERSION DES PARAMÈTRES
    
    
    if periodicite == "mensuelle":
        p = 12
    elif periodicite == "trimestrielle":
        p = 4
    elif periodicite == "semestrielle":
        p = 2
    elif periodicite == "annuelle":
        p = 1
    
    taux_periodique = r / p
    
   
    
    # 'n' est la durée de remboursement en années 
    nb_periodes_remboursement = int(n * p) 
    
    # 'differe' est en périodes  
    differe_periodes_effectif = int(differe) - 1
    
    # La durée totale est la somme des deux
    nb_periodes_total = differe_periodes_effectif + nb_periodes_remboursement
    
    tableau_data = []
    capital_restant = float(C)

    
    # ÉTAPE 2 : GESTION DE LA PHASE DE DIFFÉRÉ D'AMORTISSEMENT
    
    for i in range(1, differe_periodes_effectif + 1):
        capital_debut_periode = capital_restant 
        interets = capital_debut_periode * taux_periodique
        
        tableau_data.append({
            'Période': i,
            'Annuité': 0.0,
            'Intérêts': interets,
            'Amortissement': 0.0,
            'Capital restant': capital_debut_periode 
        })
        
        capital_restant += interets  

    capital_a_rembourser = capital_restant
    

    # ÉTAPE 3 : GESTION DE LA PHASE DE REMBOURSEMENT
    
    
    # Annuités constantes 
    if type == "anncst":
        annuite = capital_a_rembourser * taux_periodique / (1 - (1 + taux_periodique)**-nb_periodes_remboursement)
        
        for i in range(differe_periodes_effectif + 1, nb_periodes_total + 1):
            capital_debut_periode = capital_restant
            interets = capital_debut_periode * taux_periodique
            amortissement = annuite - interets
            
            tableau_data.append({
                 'Période': i, 'Annuité': annuite, 'Intérêts': interets,
                'Amortissement': amortissement, 
                'Capital restant': capital_debut_periode
            })
            
            capital_restant -= amortissement

    # Amortissements constants
    elif type == "amocst":
        amortissement_cst = capital_a_rembourser / nb_periodes_remboursement 
        
        for i in range(differe_periodes_effectif + 1, nb_periodes_total + 1):
            capital_debut_periode = capital_restant
            interets = capital_debut_periode * taux_periodique
            annuite = amortissement_cst + interets
            
            tableau_data.append({
                'Période': i, 'Annuité': annuite, 'Intérêts': interets,
                'Amortissement': amortissement_cst, 
                'Capital restant': capital_debut_periode
            })
            
            capital_restant -= amortissement_cst

    # Remboursement In fine 
    elif type == "infine":
        for i in range(differe_periodes_effectif + 1, nb_periodes_total + 1):
            capital_debut_periode = capital_restant
            interets = capital_debut_periode * taux_periodique
            amortissement = 0.0
            
            if i == nb_periodes_total: 
                amortissement = capital_a_rembourser
            
            annuite = interets + amortissement
            
            tableau_data.append({
                'Période': i, 'Annuité': annuite, 'Intérêts': interets,
                'Amortissement': amortissement, 
                'Capital restant': capital_debut_periode
            })
            
            capital_restant -= amortissement

    # Annuités en progression géométrique 
    elif type == "annprog":
        if q == 1 + taux_periodique:
            a1 = capital_a_rembourser / nb_periodes_remboursement * (1 + taux_periodique)
        else:
            a1 = capital_a_rembourser * (1 + taux_periodique - q) / (1 - (q / (1 + taux_periodique))**nb_periodes_remboursement)
        
        annuite_courante = a1
        for i in range(differe_periodes_effectif + 1, nb_periodes_total + 1):
            capital_debut_periode = capital_restant
            interets = capital_debut_periode * taux_periodique
            amortissement = annuite_courante - interets

            tableau_data.append({
                'Période': i, 'Annuité': annuite_courante, 'Intérêts': interets,
                'Amortissement': amortissement, 
                'Capital restant': capital_debut_periode
            })
            
            capital_restant -= amortissement
            annuite_courante *= q

    
    # ÉTAPE 4 : RETOUR DU DATAFRAME
    
    df = pd.DataFrame(tableau_data)
    
    return df

In [2]:
# Exemple d'utilisation pour un prêt immobilier
# Capital: 200 000€, Durée: 15 ans, Taux: 3.5%, Type: Annuités constantes, Périodicité: Mensuelle
resultat = tabamortissement(200000, 15, 0.035, "anncst", "mensuelle")

# Affichage des 10 premières lignes pour démonstration
print("Aperçu du tableau d'amortissement :")
display(resultat.head(10))

Aperçu du tableau d'amortissement :


,Période,Annuité,Intérêts,Amortissement,Capital restant
0,1,1429.765083,583.333333,846.431749,200000.000000
1,2,1429.765083,580.864574,848.900509,199153.568251
2,3,1429.765083,578.388614,851.376468,198304.667742
3,4,1429.765083,575.905433,853.859650,197453.291274
4,5,1429.765083,573.415009,856.350074,196599.431624
5,6,1429.765083,570.917321,858.847761,195743.081550
6,7,1429.765083,568.412349,861.352734,194884.233788
7,8,1429.765083,565.900070,863.865013,194022.881054
8,9,1429.765083,563.380463,866.384619,193159.016041
9,10,1429.765083,560.853508,868.911574,192292.631422
